# Modelling and Evaluation Notebook

## Objectives

* Load the cherry-leaf dataset (train / validation / test) and set up image augmentation for training.
* Save the label mapping.
* Train a small CNN 16 times at batch size 16, sweeping:
    - dropout: 0.5 or 0.3
    - kernel size: 3 or 5
    - activation: relu or elu
    - optimizer: adam or adamax
* Use EarlyStopping and save the best model for each run.
* Evaluate each run on the validation set and collect: loss, accuracy, precision, recall, F1, and a confusion matrix.
* Build a simple leaderboard to compare runs and pick the top candidates for the next step.

## Inputs

* inputs/mildew-dataset/cherry-leaves/train
* inputs/mildew-dataset/cherry-leaves/test
* inputs/mildew-dataset/cherry-leaves/validation
* image shape embeddings

## Outputs
* class_indices.pkl — saved label mapping.
* For each trained model:
    - best model file
    - training history
    - learning curves plot
    - validation metrics
* Leaderboard CSV: reports/grid_report_bs16.csv (sorted by validation accuracy, with validation loss as tie-breaker).

## Additional Comments | Insights | Conclusions

---

## Import Libraries

In [5]:

import os, math, json, itertools, time, joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.image import imread
from pathlib import Path

# TensorFlow and Keras:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Sklearn:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

---

## Set Seed

In [6]:
SEED = 27
np.random.seed(SEED)
tf.random.set_seed(SEED)

---

## Define Main Variables

In [7]:
version = 'step_1'

# Set batch size
batch_size = 16

# Set number of epochs
epochs = 25

# Define Hyperparameter Grid:
HYP_GRID = {
    "dropout":     [0.5, 0.3],
    "kernel_size": [3, 5],
    "activation":  ["relu", "elu"],
    "optimizer":   ["adam", "adamax"],
}

---

## Set Scenarios:

In [8]:
SCENARIOS = [dict(zip(HYP_GRID.keys(), vals)) for vals in itertools.product(*HYP_GRID.values())]
print(f"Total scenarios: {len(SCENARIOS)}")
SCENARIOS[:3]

Total scenarios: 16


[{'dropout': 0.5, 'kernel_size': 3, 'activation': 'relu', 'optimizer': 'adam'},
 {'dropout': 0.5,
  'kernel_size': 3,
  'activation': 'relu',
  'optimizer': 'adamax'},
 {'dropout': 0.5, 'kernel_size': 3, 'activation': 'elu', 'optimizer': 'adam'}]

### Create Run Tag Hashes 

In [9]:
def run_tag(cfg: dict, version: str, batch_size: int, seed: int | None = None, include_seed: bool = True) -> str:
    """
    Build a readable tag like:
      step_1_bs16_k3_do0.5_act-relu_opt-adam_seed27
    """
    parts = [
        version,
        f"bs{batch_size}",
        f"k{cfg['kernel_size']}",
        f"do{cfg['dropout']}",
        f"act-{cfg['activation']}",
        f"opt-{cfg['optimizer']}",
    ]
    if include_seed and seed is not None:
        parts.append(f"seed{seed}")
    return "_".join(parts)

---

# Set Directories

  ## Set Working Directory

In [10]:
# Parent directory
parent_dir =  "/Users/marcelldemeter/GIT/CodeInstitute/ci-p5-mildew-detector"

# Change working directory to parent directory
os.chdir(parent_dir)
print (f"New working directory: {os.getcwd()} ")

New working directory: /Users/marcelldemeter/GIT/CodeInstitute/ci-p5-mildew-detector 


## Set Input Directory

In [11]:
dataset_dir = "inputs/mildew-dataset/cherry-leaves"
train_dir = os.path.join(parent_dir, dataset_dir, "train")
validation_dir = os.path.join(parent_dir, dataset_dir, "validation")
test_dir = os.path.join(parent_dir, dataset_dir, "test")

## Set Output Directory

In [12]:

file_path = f'outputs/{version}'

# Create main output directory if it doesn’t exist
if 'outputs' in os.listdir(parent_dir) and version in os.listdir(parent_dir + '/outputs'):
    print('Old version is already available create a new version.')
    pass
else:
    os.makedirs(name=file_path)

# Define subfolders and ensure they exist:
models_dir = os.path.join(file_path, 'models')
reports_dir = os.path.join(file_path, 'reports')

os.makedirs(models_dir, exist_ok=True)
os.makedirs(reports_dir, exist_ok=True)


### Get Artifact Paths:

In [13]:
def get_artifact_paths(models_dir: str, reports_dir: str, tag: str) -> dict:
    """
    Return file paths (no new dirs) for this run's artifacts, using the tag in filenames.
    """
    return {
        "model_path":   Path(models_dir)  / f"{tag}.keras",
        "history_pkl":  Path(reports_dir) / f"history_{tag}.pkl",
        "evaluation_pkl": Path(reports_dir) / f"eval_{tag}.pkl",
        "curve_png":    Path(reports_dir) / f"curves_{tag}.png",
    }

---

## Set Labels

In [14]:
# Set the labels
labels = os.listdir(train_dir)
print('Label for the images are', labels)

Label for the images are ['powdery_mildew', 'healthy']


## Set image shape

In [15]:
## Import saved image shape embedding
import joblib
version = 'v1'
image_shape = joblib.load(filename=f"outputs/{version}/image_shape.pkl")
image_shape

(256, 256, 3)

---

# Image Data Augmentation

### Image Data Generator

In [16]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Initialize ImageDataGenerator

In [17]:
augmented_image_data = ImageDataGenerator(rotation_range=20,
                                          width_shift_range=0.10,
                                          height_shift_range=0.10,
                                          shear_range=0.1,
                                          zoom_range=0.1,
                                          horizontal_flip=True,
                                          vertical_flip=True,
                                          fill_mode='nearest',
                                          rescale=1./255
                                          )


### Augment image datasets

In [18]:
# Train Set:
train_set = augmented_image_data.flow_from_directory(train_dir,
                                                     target_size=image_shape[:2],
                                                     color_mode='rgb',
                                                     batch_size=batch_size,
                                                     class_mode='binary',
                                                     shuffle=True
                                                     )

# Validation Set:
validation_set = ImageDataGenerator(rescale=1./255).flow_from_directory(validation_dir,
                                                                        target_size=image_shape[:2],
                                                                        color_mode='rgb',
                                                                        batch_size=batch_size,
                                                                        class_mode='binary',
                                                                        shuffle=False
                                                                        )

# Test Set:
test_set = ImageDataGenerator(rescale=1./255).flow_from_directory(test_dir,
                                                                  target_size=image_shape[:2],
                                                                  color_mode='rgb',
                                                                  batch_size=batch_size,
                                                                  class_mode='binary',
                                                                  shuffle=False
                                                                  )


train_set.class_indices
validation_set.class_indices
test_set.class_indices

Found 2944 images belonging to 2 classes.
Found 420 images belonging to 2 classes.
Found 844 images belonging to 2 classes.


{'healthy': 0, 'powdery_mildew': 1}

### Save class_indices

In [19]:
joblib.dump(value=train_set.class_indices,
            filename=f"{file_path}/class_indices.pkl")

['outputs/step_1/class_indices.pkl']

---

# Model Creation

## ML Model

### Model Builder:

In [20]:

def build_model(cfg):
    """
    CNN builder using the scenario dict `cfg`.
    Expects keys: 'kernel_size', 'dropout', 'activation'.
    Uses global `image_shape`.
    """
    k   = int(cfg["kernel_size"])
    do  = float(cfg["dropout"])
    act = str(cfg["activation"])

    m = Sequential(name=f"cnn_k{k}_do{do}_{act}")
    m.add(Conv2D(32, (k, k), padding='same', activation=act, input_shape=image_shape))
    m.add(BatchNormalization()); m.add(MaxPooling2D((2,2)))

    m.add(Conv2D(64, (k, k), padding='same', activation=act))
    m.add(BatchNormalization()); m.add(MaxPooling2D((2,2)))

    m.add(Conv2D(64, (k, k), padding='same', activation=act))
    m.add(BatchNormalization()); m.add(MaxPooling2D((2,2)))

    m.add(Flatten())
    m.add(Dropout(do))
    m.add(Dense(64, activation=act))
    m.add(Dropout(do))
    m.add(Dense(1, activation='sigmoid'))
    return m


### Save History Plot

In [21]:
def save_history_plot(history, out_png):
    h = history.history
    plt.figure(figsize=(8,5))
    if "accuracy" in h and "val_accuracy" in h:
        plt.plot(h["accuracy"], label="train_acc")
        plt.plot(h["val_accuracy"], label="val_acc")
    if "loss" in h and "val_loss" in h:
        plt.plot(h["loss"], label="train_loss")
        plt.plot(h["val_loss"], label="val_loss")
    plt.xlabel("Epoch"); plt.ylabel("Value"); plt.legend(); plt.tight_layout()
    plt.savefig(out_png, dpi=120); plt.close()

### Train and Evaluate Model

In [22]:
def train_and_eval(cfg: dict):
    # tag + artifact paths
    tag = run_tag(cfg, version, batch_size, seed=SEED)
    P = get_artifact_paths(models_dir, reports_dir, tag)

    # build & compile model
    model = build_model(cfg)
    model.compile(
        optimizer=cfg["optimizer"],
        loss="binary_crossentropy",
        metrics=[tf.keras.metrics.BinaryAccuracy(name="accuracy")]
    )

    # callbacks (save best by val_accuracy)
    cbs = [
        # Stop training if val_accuracy doesn't improve for 4 epochs, roll back to best weights:
        EarlyStopping(monitor="val_accuracy", mode="max", patience=4, restore_best_weights=True, verbose=1),
        # Keep the best model:
        ModelCheckpoint(filepath=str(P["model_path"]), monitor="val_accuracy", mode="max", save_best_only=True, verbose=1),
    ]

    # Fit Model
    history = model.fit(
        train_set,
        epochs= epochs,
        batch_size=batch_size,
        validation_data=validation_set,
        callbacks=cbs,
        verbose=2
    )

    # Save history + curves
    joblib.dump(history.history, P["history_pkl"])
    save_history_plot(history, P["curve_png"])

    # Evaluate on validation: get probabilities, true labels, and hard preds
    validation_set.reset()                                      # start from first batch, keep order aligned
    y_prob = model.predict(validation_set, verbose=0).squeeze()  # (N,)
    y_true = validation_set.classes                              # (N,)
    y_pred = (y_prob >= 0.5).astype(int)                         # threshold @ 0.5

    # Capture val_loss and Keras accuracy for the same validation set ---
    validation_set.reset()                                       # reset again before evaluate
    val_loss, val_acc = model.evaluate(validation_set, verbose=0) # loss & accuracy from the compiled metrics

    # Compute metrics
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec  = recall_score(y_true, y_pred, zero_division=0)
    f1   = f1_score(y_true, y_pred, zero_division=0)
    cm   = confusion_matrix(y_true, y_pred).tolist()

    # --- Summary, CSV-ready ---
    out = {
        "tag": tag,                          # Unique ID
        "version": version,
        "batch_size": batch_size,
        **cfg,                               # includes dropout, kernel_size, activation, optimizer
        "epochs_trained": len(history.history["loss"]),
        "val_loss": float(val_loss),
        "val_accuracy": float(val_acc),
        "val_precision": float(prec),
        "val_recall": float(rec),
        "val_f1": float(f1),
        "confusion_matrix": cm,
    }

    # Save Evaluation Pickle
    joblib.dump(out, P["evaluation_pkl"])

    # Hand back the summary so the caller can append to a list/DataFrame
    return out


### Execute Scenarios:

In [23]:
results = []
t0 = time.time()
for i, cfg in enumerate(SCENARIOS, 1):
    print(f"\n=== [{i}/{len(SCENARIOS)}] {cfg} ===")
    res = train_and_eval(cfg)          # uses your build_model + callbacks + eval
    results.append(res)

print(f"\nCompleted {len(SCENARIOS)} runs in ~{(time.time()-t0)/60:.1f} min")


=== [1/16] {'dropout': 0.5, 'kernel_size': 3, 'activation': 'relu', 'optimizer': 'adam'} ===


/Users/marcelldemeter/GIT/CodeInstitute/ci-p5-mildew-detector/mildew-env/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-10-12 01:20:55.308694: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-10-12 01:20:55.308737: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-10-12 01:20:55.308751: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.66 GB
2025-10-12 01:20:55.308772: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-10-12 01:20:55.308784: I tensorflow/core/common_r

Epoch 1/25


2025-10-12 01:20:56.011113: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.



Epoch 1: val_accuracy improved from None to 0.50000, saving model to outputs/step_1/models/v1_bs16_k3_do0.5_act-relu_opt-adam_seed27.keras
184/184 - 21s - 116ms/step - accuracy: 0.9609 - loss: 2.1693 - val_accuracy: 0.5000 - val_loss: 98.5428
Epoch 2/25

Epoch 2: val_accuracy did not improve from 0.50000
184/184 - 18s - 96ms/step - accuracy: 0.9647 - loss: 3.9049 - val_accuracy: 0.5000 - val_loss: 74.7050
Epoch 3/25

Epoch 3: val_accuracy improved from 0.50000 to 0.52857, saving model to outputs/step_1/models/v1_bs16_k3_do0.5_act-relu_opt-adam_seed27.keras
184/184 - 18s - 95ms/step - accuracy: 0.9698 - loss: 3.2349 - val_accuracy: 0.5286 - val_loss: 53.2930
Epoch 4/25

Epoch 4: val_accuracy improved from 0.52857 to 0.99048, saving model to outputs/step_1/models/v1_bs16_k3_do0.5_act-relu_opt-adam_seed27.keras
184/184 - 18s - 99ms/step - accuracy: 0.9749 - loss: 3.6338 - val_accuracy: 0.9905 - val_loss: 1.1243
Epoch 5/25

Epoch 5: val_accuracy did not improve from 0.99048
184/184 - 18s 

...

Epoch 1/25

Epoch 1: val_accuracy improved from None to 0.50000, saving model to outputs/step_1/models/v1_bs16_k5_do0.3_act-relu_opt-adamax_seed27.keras
184/184 - 24s - 132ms/step - accuracy: 0.9701 - loss: 0.7864 - val_accuracy: 0.5000 - val_loss: 11.9052
Epoch 2/25

Epoch 2: val_accuracy improved from 0.50000 to 0.50238, saving model to outputs/step_1/models/v1_bs16_k5_do0.3_act-relu_opt-adamax_seed27.keras
184/184 - 21s - 116ms/step - accuracy: 0.9806 - loss: 0.4304 - val_accuracy: 0.5024 - val_loss: 23.9806
Epoch 3/25

Epoch 3: val_accuracy improved from 0.50238 to 0.90476, saving model to outputs/step_1/models/v1_bs16_k5_do0.3_act-relu_opt-adamax_seed27.keras
184/184 - 21s - 116ms/step - accuracy: 0.9796 - loss: 0.4750 - val_accuracy: 0.9048 - val_loss: 1.8782
Epoch 4/25

Epoch 4: val_accuracy improved from 0.90476 to 0.99286, saving model to outputs/step_1/models/v1_bs16_k5_do0.3_act-relu_opt-adamax_seed27.keras
184/184 - 21s - 117ms/step - accuracy: 0.9817 - loss: 0.6450 - val_a

/Users/marcelldemeter/GIT/CodeInstitute/ci-p5-mildew-detector/mildew-env/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/25

Epoch 1: val_accuracy improved from None to 0.90000, saving model to outputs/step_1/models/v1_bs16_k5_do0.3_act-elu_opt-adam_seed27.keras
184/184 - 34s - 185ms/step - accuracy: 0.9406 - loss: 0.5673 - val_accuracy: 0.9000 - val_loss: 0.4797
Epoch 2/25

Epoch 2: val_accuracy improved from 0.90000 to 0.96429, saving model to outputs/step_1/models/v1_bs16_k5_do0.3_act-elu_opt-adam_seed27.keras
184/184 - 32s - 174ms/step - accuracy: 0.9548 - loss: 0.2178 - val_accuracy: 0.9643 - val_loss: 0.1043
Epoch 3/25

Epoch 3: val_accuracy improved from 0.96429 to 0.96905, saving model to outputs/step_1/models/v1_bs16_k5_do0.3_act-elu_opt-adam_seed27.keras
184/184 - 31s - 171ms/step - accuracy: 0.9738 - loss: 0.1086 - val_accuracy: 0.9690 - val_loss: 0.0944
Epoch 4/25

Epoch 4: val_accuracy improved from 0.96905 to 0.97857, saving model to outputs/step_1/models/v1_bs16_k5_do0.3_act-elu_opt-adam_seed27.keras
184/184 - 32s - 174ms/step - accuracy: 0.9840 - loss: 0.0850 - val_accuracy: 0.978

/Users/marcelldemeter/GIT/CodeInstitute/ci-p5-mildew-detector/mildew-env/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/25

Epoch 1: val_accuracy improved from None to 0.50000, saving model to outputs/step_1/models/v1_bs16_k5_do0.3_act-elu_opt-adamax_seed27.keras
184/184 - 38s - 204ms/step - accuracy: 0.9660 - loss: 0.5283 - val_accuracy: 0.5000 - val_loss: 23.9374
Epoch 2/25

Epoch 2: val_accuracy did not improve from 0.50000
184/184 - 44s - 240ms/step - accuracy: 0.9789 - loss: 0.0832 - val_accuracy: 0.5000 - val_loss: 8.9780
Epoch 3/25

Epoch 3: val_accuracy improved from 0.50000 to 0.95000, saving model to outputs/step_1/models/v1_bs16_k5_do0.3_act-elu_opt-adamax_seed27.keras
184/184 - 43s - 235ms/step - accuracy: 0.9885 - loss: 0.0430 - val_accuracy: 0.9500 - val_loss: 0.1366
Epoch 4/25

Epoch 4: val_accuracy improved from 0.95000 to 0.99762, saving model to outputs/step_1/models/v1_bs16_k5_do0.3_act-elu_opt-adamax_seed27.keras
184/184 - 46s - 250ms/step - accuracy: 0.9925 - loss: 0.0343 - val_accuracy: 0.9976 - val_loss: 0.0067
Epoch 5/25

Epoch 5: val_accuracy did not improve from 0.99762

### Create Dataframe on results and save to .csv

In [24]:
df = pd.DataFrame(results).sort_values(
    by=["val_accuracy", "val_loss"], ascending=[False, True] # Sort by highest accuracy, then lowest loss
).reset_index(drop=True)

report_csv = os.path.join(reports_dir, f"grid_report_bs{batch_size}.csv")
df.to_csv(report_csv, index=False)
print(f"Saved report to: {report_csv}")

Saved report to: outputs/step_1/reports/grid_report_bs16.csv


### See Results

In [25]:
df.head(8) # Display top 8 results

,tag,version,batch_size,dropout,kernel_size,activation,optimizer,epochs_trained,val_loss,val_accuracy,val_precision,val_recall,val_f1,confusion_matrix
0,v1_bs16_k5_do0.3_act-relu_opt-adamax_seed27,v1,16,0.3,5,relu,adamax,10,0.000000,1.000000,1.000000,1.000000,1.000000,"[[210, 0], [0, 210]]"
1,v1_bs16_k3_do0.3_act-elu_opt-adamax_seed27,v1,16,0.3,3,elu,adamax,11,0.000965,1.000000,1.000000,1.000000,1.000000,"[[210, 0], [0, 210]]"
2,v1_bs16_k5_do0.5_act-elu_opt-adamax_seed27,v1,16,0.5,5,elu,adamax,12,0.002597,1.000000,1.000000,1.000000,1.000000,"[[210, 0], [0, 210]]"
3,v1_bs16_k5_do0.3_act-elu_opt-adamax_seed27,v1,16,0.3,5,elu,adamax,8,0.006746,0.997619,0.995261,1.000000,0.997625,"[[209, 1], [0, 210]]"
4,v1_bs16_k3_do0.3_act-relu_opt-adamax_seed27,v1,16,0.3,3,relu,adamax,9,0.038951,0.997619,1.000000,0.995238,0.997613,"[[210, 0], [1, 209]]"
5,v1_bs16_k3_do0.5_act-elu_opt-adam_seed27,v1,16,0.5,3,elu,adam,10,0.064844,0.997619,1.000000,0.995238,0.997613,"[[210, 0], [1, 209]]"
6,v1_bs16_k3_do0.5_act-relu_opt-adamax_seed27,v1,16,0.5,3,relu,adamax,7,0.229929,0.997619,1.000000,0.995238,0.997613,"[[210, 0], [1, 209]]"
7,v1_bs16_k3_do0.3_act-relu_opt-adam_seed27,v1,16,0.3,3,relu,adam,12,0.233087,0.997619,0.995261,1.000000,0.997625,"[[209, 1], [0, 210]]"
